# Report – Scene “Dialoghi con un’Eco” (Hardening of **ENTITÀ**)

## Goal

Reduce out-of-context and cliché outputs from ENTITÀ, anchoring its speech to the current IO line and making it speak only when meaningful.
All adjustments were done locally in the scene, without modifying the global `EntityBrain`.

---

## What We Implemented (Technical Summary)

* Robust context extraction and cleanup (handles dicts/tuples/objects, strips prefixes like “IO:”).
* Hard focus: ENTITÀ must answer the last meaningful line (preferably by IO).
  If the context is too weak → outputs `SILENCE` and does not display overlay.
* Local “red-teamed” prompt: one sentence (10–14 words), no prefixes/quotes, must include at least one keyword from recent context.
* Dynamic keywording from the last 12 lines.
* On-scene gating and re-ranking:

  * must contain ≥1 focus term and ≥1 context keyword;
  * minimum Jaccard overlap with context;
  * anti-repetition check;
  * banlist of overused or generic words.
* Noise reduction: `temperature = 0.35`, ENTITÀ triggers only after an IO line.
* Debug tracing: logs every rejection reason (low overlap, no keyword, banlist hit, etc.).

---

## What the Log Shows

### 1) Activation pattern

The controller oscillates between speak=False/True, confirming that the decision gate is active.
→ Desired behavior: ENTITÀ does not speak at every frame anymore.

### 2) Compliance with formal rules

* Sentences are short (≈10–12 words).
* Correct Italian, no prefixes like “ENTITÀ:”, ends with a period.
* Each output includes at least one keyword from context.

Structural constraints respected consistently.

### 3) Semantic anchoring to the focus line

* Still weak in early sections:
  With focus “che inducono la mente umana a smascherarsi”, outputs like
  “L’uomo inganna spesso se stesso con pensieri propri e bugie quotidiane.”
  are formally correct but context-neutral (no reuse of *mente*, *smascherarsi*).
* Much stronger later (focus “bypassando le difese imposte”):
  “Le difese vengono facilmente bypassate dalla mente umana.” → clear contextual link.

Anchoring works, but focus tokens should become mandatory.

### 4) Repetitions and clichés

* Near-duplicates such as
  “L’uomo inganna se stesso...” repeat across calls.
* Clichés remain (“fragili difese”, “vera coscienza interiore”).

Anti-repetition threshold and banlist need tightening.

### 5) Instruction artefacts

Some outputs literally include the word “punto” (“... viene il caos punto.”)
→ misinterprets the rule “end with a period”.

Need an explicit clause: “Do not write the word ‘punto’” and a post-edit cleanup.

---

## Overall Evaluation

**Improvements achieved**

* ENTITÀ speaks less and more purposefully.
* Outputs are formally compliant and syntactically clean.
* Stronger contextual link in mid-scene (“difese imposte”).

**Remaining issues**

* Focus leakage in early dialogue (not reusing focus tokens).
* Mild repetition of similar phrasings.
* Residual clichés and the “punto” artefact.

Partial success: behavior is controlled, cleaner, and occasionally well-anchored.

---

## Corrective Actions (Next Iteration)

1. Focus-token enforcement – require ≥1 lemma from the focus line; fallback to *SILENCE* otherwise.
2. Higher Jaccard threshold (0.08 → 0.12–0.15) + penalize generic adverbs (“sempre”, “costantemente”).
3. Stronger anti-repetition – 4-5-gram blocking + local cache penalty.
4. Extended cliché banlist – *fragile, interiore, vera, oscurità, abisso* (unless high overlap).
5. Remove “punto” artefact – add explicit rule + post-edit substitution.
6. Sampling tuning – `temperature=0.25–0.30`, `top_p=0.8`, `repetition_penalty≈1.1`.
7. Scene-level tests – 5–10 sample turns; track focus-token %, overlap %, diversity, and silence ratio.

---

## Examples (from the log)

| Type       | Example                                                                          | Notes                        |
| ---------- | -------------------------------------------------------------------------------- | ---------------------------- |
| Generic    | “L’uomo inganna spesso se stesso con pensieri propri e bugie quotidiane.”        | well-formed; no focus tokens |
| Anchored   | “Le difese vengono facilmente bypassate dalla mente umana.”                      | uses focus terms             |
| Cliché     | “L’uomo costruisce difese per nascondere la sua vera coscienza interiore punto.” | cliché + “punto” artefact    |
| Repetition | “L’uomo inganna se stesso...” (reappears in 3 calls)                             | needs stronger blocking      |

---

## Conclusion

The hardening phase is a technical success: ENTITÀ now produces structured, infrequent, context-aware responses instead of random monologues.
However, for full contextual fidelity we still need to enforce focus reuse, aggressive anti-repetition, and cliché filtering.
After those, ENTITÀ should become precisely reactive, genuinely tied to IO’s line, and silence itself when no relevant context exists.
